In [ ]:
import wazy
import jax
import jax.numpy as jnp
import random
import numpy as np
import tensorflow as tf
import urllib

# import tensorflowjs as tfjs
import json

In [ ]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], "GPU")
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != "GPU"
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [ ]:
alphabet = [
    "A",
    "R",
    "N",
    "D",
    "C",
    "Q",
    "E",
    "G",
    "H",
    "I",
    "L",
    "K",
    "M",
    "F",
    "P",
    "S",
    "T",
    "W",
    "Y",
    "V",
    "B",
    "Z",
    "X",
    "*",
]
alphabet_short = alphabet[:20]
urllib.request.urlretrieve(
    "https://github.com/ur-whitelab/peptide-dashboard/raw/master/models/hemo-rnn/keras_model/model_weights.h5",
    "model_weights.h5",
)
urllib.request.urlretrieve(
    "https://github.com/ur-whitelab/peptide-dashboard/raw/master/models/hemo-rnn/keras_model/model.json",
    "model.json",
)
# load json and create model
json_file = open("model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
predict_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
predict_model.load_weights("model_weights.h5")
print("Loaded model from disk.")


def vectorize(seq):
    return np.array([[alphabet.index(s) + 1 for s in list(seq)]])


def label(seq):
    vectorized_seq = vectorize(seq)
    # function counts_aa obtains amino acid counts frequency vector
    def counts_aa(vec):
        counts = tf.histogram_fixed_width(vec, [0, 20], nbins=21)[1:]
        return counts / tf.reduce_sum(counts)

    vectorized_seq_fr = counts_aa(vectorized_seq)[tf.newaxis, ...]
    # inputs.shape[-1] needs to be 190, so we pad zeros to the end
    vectorized_seq = np.concatenate(
        [
            vectorized_seq,
            np.zeros((vectorized_seq.shape[0], 190 - vectorized_seq.shape[-1])),
        ],
        axis=-1,
    )
    y_predict = predict_model.predict([vectorized_seq, vectorized_seq_fr], verbose=0)
    return np.squeeze(y_predict)


def random_seq_generator(length):
    seq = [random.choice(Alphabet_short) for i in range(length)]
    return "".join(seq)


label("TEST")

In [ ]:
from tester import test

test(label, 20, 5, 13)